# Jedha Kayak project #block1
Notebook for development

In [111]:
# import
import os
import uuid
import requests
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display
from geopy.geocoders import Nominatim

In [70]:
# env variables
load_dotenv()
OPEN_WEATHER_TOKEN = os.getenv('Open_weather_token')

## 1. Get weather data

In [71]:
french_top_35 = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre",
                "Rouen", "Paris", "Amiens", "Lille", "Strasbourg",
                "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim",
                "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon",
                "Gorges du Verdon", "Bormes les Mimosas", "Cassis",
                "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes",
                "Aigues Mortes", "Saintes Maries de la mer", "Collioure",
                "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz",
                "Bayonne", "La Rochelle"]

In [78]:
# init dataframe
weather_df = pd.DataFrame(french_top_35, columns=["cities"])
display(weather_df.sample(2))

,cities
13,Dijon
21,Aix en Provence


In [101]:
# coordinates of cities
def cities_to_coords(city_name) :
    """ get cities coordiniates.
    """

    # geocoder
    geolocator = Nominatim(user_agent="app")
    location = geolocator.geocode(city_name)

    # full_address, latitude, longitude
    full_address = location.address
    latitude = location.latitude
    longitude = location.longitude

    # return 
    return pd.Series([full_address, latitude, longitude])

## RUN
if not "full_address" in weather_df.columns :
    weather_df[["full_address", "latitude", "longitude"]] = weather_df["cities"].apply(cities_to_coords)
    display(weather_df.sample(2))

else : 
    display(weather_df.sample(2))
    print("full_address, latitude & longitude columns exists !")

,cities,full_address,latitude,longitude
32,Biarritz,"Biarritz, Bayonne, Pyrénées-Atlantiques, Nouve...",43.471144,-1.552727
8,Strasbourg,"Strasbourg, Bas-Rhin, Grand Est, France métrop...",48.584614,7.750713


In [121]:
def volume_rain(lat, lon, exclude, appid) :
    """ volume of rain in each city over 7 next days.
    """

    # open weather one call api
    params = (
        ('lat', lat),
        ('lon', lon),
        ('exclude', exclude),
        ('appid', appid)
    )

    response = requests.get('https://api.openweathermap.org/data/2.5/onecall', params=params)

    # normalize reponse
    open_weather_7days = pd.json_normalize(response.json()["daily"])

    # clean rain column
    open_weather_7days["rain"] = open_weather_7days["rain"].apply(lambda x : 0 if np.isnan(x) else x) 

    # calculate volumn 
    n_days = open_weather_7days.shape[0]
    volume_rain_7days =  np.sum(open_weather_7days["rain"] * open_weather_7days["pop"]) / n_days

    # return 
    return round(volume_rain_7days,3)

In [108]:
# fill weather_df
exclude = 'current,minutely,hourly,alerts'
appid = OPEN_WEATHER_TOKEN
if not "volume_rain_7days" in weather_df.columns :
    weather_df["volume_rain_7days"] = weather_df[["latitude", "longitude"]].transpose().apply(lambda x : volume_rain(x[0], x[1], exclude, appid))

display(weather_df.sample(2))

,cities,full_address,latitude,longitude,volume_rain_7days
8,Strasbourg,"Strasbourg, Bas-Rhin, Grand Est, France métrop...",48.584614,7.750713,3.694
21,Aix en Provence,"Aix-en-Provence, Bouches-du-Rhône, Provence-Al...",43.529842,5.447474,1.273


In [120]:
# unique identifier (uuid)
if not "uuid" in weather_df.columns :
    weather_df['uuid'] = weather_df.index.to_series().map(lambda x: uuid.uuid4())

display(weather_df.sample(2))

,uuid,cities,full_address,latitude,longitude,volume_rain_7days
14,83a02809-a30e-4b83-9f60-36ca5b95f1d9,Annecy,"Annecy, Haute-Savoie, Auvergne-Rhône-Alpes, Fr...",45.899235,6.128885,5.134
29,1abe230d-0f14-463c-a0a0-b55ae1c77eb7,Ariege,"Ariège, Occitanie, France métropolitaine, France",42.945537,1.406554,3.691


In [119]:
# reorder columns
keep_col = ['uuid', 'cities', 'full_address', 'latitude', 'longitude', 'volume_rain_7days']
weather_df = weather_df[keep_col]

display(weather_df.sample(2))

,uuid,cities,full_address,latitude,longitude,volume_rain_7days
30,5b3627cb-d259-46d6-884e-7673ef690e14,Toulouse,"Toulouse, Haute-Garonne, Occitanie, France mét...",43.604462,1.444247,3.405
17,7aa00601-bf8e-4d74-a566-9db3397df918,Gorges du Verdon,"Gorges du Verdon, Route des Crêtes, Les Ferrai...",43.749656,6.328562,0.376


In [122]:
# save dataframe
weather_data = "../data/temp/weather_data.csv"
weather_df.to_csv(weather_data)

In [124]:
# plot best destinations
top_5_destinations = weather_df.sort_values(by=['volume_rain_7days'])[:5]
top_5_destinations


,uuid,cities,full_address,latitude,longitude,volume_rain_7days
27,74fb4f64-c63a-4274-a044-f7f3b5bd3977,Collioure,"Collioure, Céret, Pyrénées-Orientales, Occitan...",42.525050,3.083155,0.082
17,7aa00601-bf8e-4d74-a566-9db3397df918,Gorges du Verdon,"Gorges du Verdon, Route des Crêtes, Les Ferrai...",43.749656,6.328562,0.376
18,d9dd8ba3-5bf1-45e2-883d-c08aed96e844,Bormes les Mimosas,"Bormes-les-Mimosas, Toulon, Var, Provence-Alpe...",43.157217,6.329254,0.665
24,3861a591-4f3b-4414-a510-d57c4f6b23a1,Nimes,"Nîmes, Gard, Occitanie, France métropolitaine,...",43.837425,4.360069,0.954
23,e200e2d3-973b-4229-8fd9-f1b32c3d3208,Uzes,"Uzès, Nîmes, Gard, Occitanie, France métropoli...",44.012128,4.419672,1.047
